In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
Count_LS = spacex_df.groupby('Launch Site')['class'].sum()
print(Count_LS)
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site_dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}

                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site",
                                             searchable=True),


                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites

        # return the outcomes piechart for a selected site
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='successpiechart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload_slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500:'2500',5000:'5000',7500:'7500',10000:'10000'},
                                                value=[2500, 8000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='successpayloadscatterchart')),
                                ]
                      )
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='successpiechart', component_property='figure'),
             [Input(component_id='site_dropdown', component_property='value')]
)

def get_pie_chart(site_dropdown):
   filtered_df = spacex_df
   filtered_df1 = spacex_df[spacex_df['Launch Site']==site_dropdown]
   if site_dropdown == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return fig
   else:
      fig = px.pie(filtered_df1, names='class', title='Total Success Launches by '+site_dropdown)
      return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='successpayloadscatterchart', component_property='figure'),
              [Input(component_id='site_dropdown', component_property='value'),
               Input(component_id="payload_slider", component_property="value")]
)
def get_scatter_chart(site_dropdown,payload_slider):
   filtered_df = spacex_df
   filtered_df_pl = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])&(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
   filtered_df1 = filtered_df_pl[filtered_df_pl['Launch Site']==site_dropdown]
   if site_dropdown == 'ALL':
        fig = px.scatter(filtered_df_pl, x="Payload Mass (kg)", y="class", color='Booster Version Category',
                         title='Correlation between Payload and success for all Sites')
        return fig
   else:
       fig = px.scatter(filtered_df1, x="Payload Mass (kg)", y="class", color='Booster Version Category',
                        title='Correlation between Payload and success for '+site_dropdown)
       return fig
# Run the app
if __name__ == '__main__':
    app.run_server()